In [52]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *

In [53]:
# Name master に含まれているデータを選択
table_name = 'V_RESTAURANT_NAME_MASTER'
columns= "all"
# データ保存先
data_folda = '../data/'
save_columns=['restaurant_id', 'original_id', 'closed_date', 'base_date', 'target3','eval_period']

In [66]:
# snowflake から master data を読み込み
pd_sf = read_table(data_folda,table_name,columns,read_snowflake=False,col="*")

In [84]:
# 課題：将来的にはS3から取得できるようにする。
master_data=pd.read_csv(data_folda+'restaurant_data.csv')

In [85]:
# original_id, available, closed_date, closed_flg のみを抽出
pd_sf = pd_sf[['restaurant_id','original_id', 'available', 'closed_date', 'business_status',"prefecture"]]

In [93]:
# original_id is not null and available is 1 
pd_sf=pd_sf[pd_sf['original_id'].isna()==False]
pd_sf["available"]=pd_sf["available"].astype(int)
pd_sf=pd_sf[pd_sf["available"]==1]
pd_sf['original_id']=pd_sf['original_id'].astype(int)

In [94]:
# id で created_at を結合
pd_sf = pd.merge(pd_sf,master_data[['id','created_at']],left_on='original_id',right_on="id",how='left')

In [98]:
# close_date を日付に変換
pd_sf['closed_date'] = pd.to_datetime(pd_sf['closed_date'], format='%Y-%m-%d', errors='coerce')
# created_at を日付に変換
pd_sf['created_at'] = pd.to_datetime(pd_sf['created_at'], format='%Y-%m-%d', errors='coerce')

In [99]:
# yyyymmddの列を追加する。
# 他のデータの内容は変わらず、2020年1月から2023年11月まで各月分のデータを用意する。
# 日付けは各月末日とする
# pandas のconcatは時間がかかるので、一旦、別に保存して最後に一度にconcatする。

eval_period=pd.date_range('2019-01-01', '2023-12-01', freq='MS')
#eval_period から一日前の日付を取得
eval_period=eval_period - pd.Timedelta(days=1)

# len(eval_period)のサイズを持つ、pandas のリストを作成
temp_=[pd.DataFrame() for x in range(len(eval_period))]


for i in eval_period:
    print(i)
    # pd_sfのコピーを作成
    pd_sf_copy=pd_sf.copy()
    # yyyymmddの列を追加、データの受領日を本来入れるべきであるが、ここでは1日を入れる
    pd_sf_copy['base_date']=i
    # augumented_dataに追加
    temp_[eval_period.get_loc(i)]=pd_sf_copy
    
# pseudo_time_series を作成
pseudo_time_series=pd.concat([temp_[i] for i in range(len(temp_))], ignore_index=True)


2018-12-31 00:00:00
2019-01-31 00:00:00
2019-02-28 00:00:00
2019-03-31 00:00:00
2019-04-30 00:00:00
2019-05-31 00:00:00
2019-06-30 00:00:00
2019-07-31 00:00:00
2019-08-31 00:00:00
2019-09-30 00:00:00
2019-10-31 00:00:00
2019-11-30 00:00:00
2019-12-31 00:00:00
2020-01-31 00:00:00
2020-02-29 00:00:00
2020-03-31 00:00:00
2020-04-30 00:00:00
2020-05-31 00:00:00
2020-06-30 00:00:00
2020-07-31 00:00:00
2020-08-31 00:00:00
2020-09-30 00:00:00
2020-10-31 00:00:00
2020-11-30 00:00:00
2020-12-31 00:00:00
2021-01-31 00:00:00
2021-02-28 00:00:00
2021-03-31 00:00:00
2021-04-30 00:00:00
2021-05-31 00:00:00
2021-06-30 00:00:00
2021-07-31 00:00:00
2021-08-31 00:00:00
2021-09-30 00:00:00
2021-10-31 00:00:00
2021-11-30 00:00:00
2021-12-31 00:00:00
2022-01-31 00:00:00
2022-02-28 00:00:00
2022-03-31 00:00:00
2022-04-30 00:00:00
2022-05-31 00:00:00
2022-06-30 00:00:00
2022-07-31 00:00:00
2022-08-31 00:00:00
2022-09-30 00:00:00
2022-10-31 00:00:00
2022-11-30 00:00:00
2022-12-31 00:00:00
2023-01-31 00:00:00


In [100]:
# base_date を日付に変換
pseudo_time_series["base_date"]=pd.to_datetime(pseudo_time_series["base_date"], format='%Y-%m-%d', errors='coerce')

In [101]:
# target flg を追加する
# closed_date 該当月はflg=1とする
pseudo_time_series["closed_flg"]=0
pseudo_time_series.loc[pseudo_time_series["closed_date"].dt.strftime('%Y%m')==pseudo_time_series["base_date"].dt.strftime('%Y%m'),"closed_flg"]=1

In [102]:
# 同じrestaurant_idで、closed_flg=1のデータがある場合、その月以降はclosed_flg=1とする
pseudo_time_series.sort_values(["restaurant_id","base_date"],inplace=True)
pseudo_time_series["closed_flg"]=pseudo_time_series.groupby("restaurant_id")["closed_flg"].cummax()

In [103]:
# closed_flgをrestaurant_id毎に-3ヶ月ずらした列を追加する。これがtarget3になる。
pseudo_time_series["target3"]=pseudo_time_series.groupby("restaurant_id")["closed_flg"].shift(-3).ffill().astype(int)
# closed_flgをrestaurant_id毎に-6ヶ月ずらした列を追加する。これがtarget6になる。
pseudo_time_series["target6"]=pseudo_time_series.groupby("restaurant_id")["closed_flg"].shift(-6).ffill().astype(int)


In [121]:
# input は、closed_flgが1になる前の月までとする。
pseudo_time_series["eval_period"]=True
# augumented_data.loc[augumented_data.groupby("restaurant_id")["closed_flg"].shift(-1).ffill()==1,"eval_target3"]=False
# base_date 以前に閉店したものeval_periodをFalseにする。
pseudo_time_series.loc[pseudo_time_series["closed_date"].dt.date<=pseudo_time_series["base_date"],"eval_period"]=False

# created_at が base_date 以降のものeval_periodをFalseにする。
pseudo_time_series.loc[pseudo_time_series["base_date"] < pseudo_time_series["created_at"].dt.date,"eval_period"]=False

In [122]:
pseudo_time_series["closed_flg"]=pseudo_time_series["closed_flg"].astype(int)

pseudo_time_series.sort_values(["restaurant_id","base_date"],inplace=True)

# 作成した時系列データを保存する
pseudo_time_series[save_columns].to_pickle(data_folda+'skeleton_train.pkl')

* master_dataから base_date 時点で掲載中であったお店を選択する
* base_date から3ヶ月間で閉店するお店の target3=1とする。

In [123]:
base_date="2023-12-01"
# base_dateの一日前の日付を取得
base_date_minus_1day=pd.to_datetime(base_date)-pd.Timedelta(days=1)
# base_dateの3か月後の日付を取得
base_date_plus_3month=pd.to_datetime(base_date)+pd.DateOffset(months=3)
base_date_plus_3month=base_date_plus_3month.strftime('%Y-%m-%d')

# pseudo_time_seriesの中でbase_dateの一日前の日と一致するものを取得
test=pseudo_time_series[pseudo_time_series["base_date"]==base_date_minus_1day]

# master_dataのclose_date, created_atを日付に変換
master_data["close_date"]=pd.to_datetime(master_data["close_date"],errors="coerce")
master_data["created_at"]=pd.to_datetime(master_data["created_at"],errors="coerce")

# base_dateより前に掲載を開始かつ、現在掲載中の店舗のrestaurant_idを取得
nonbaias_restaurant_id_open=master_data[(master_data["business_status"]==0) 
                                        & (master_data["created_at"]<base_date)]["id"]
# base_date以降に閉店した店舗かつ、base_date以前に掲載を開始したお店のrestaurant_idを取得
nonbaias_restaurant_id_close=master_data[(master_data["business_status"]==1) 
                                         & (master_data["close_date"]>=base_date)
                                         & (master_data["created_at"]<base_date)]["id"] 
# 二つを結合
nonbaias_restaurant_id=pd.concat([nonbaias_restaurant_id_open,nonbaias_restaurant_id_close])

# testのoriginal_idがnonbaias_restaurant_idに含まれるものを取得
test=test[test["original_id"].isin(nonbaias_restaurant_id)]
# eval_periodを全てTrueにする
test.loc[test["original_id"].isin(nonbaias_restaurant_id),"eval_period"]=True

# master_data の中でbase_dateから3カ月以内にに閉店した店舗の restaurant_id を取得
closed_in_3month_from20231201=master_data[(master_data["close_date"]>=base_date)
                     &(master_data["close_date"]<base_date_plus_3month)
                     &(master_data["business_status"]==1)]["id"]

# testのoriginal_idがclosed_in_3month_from20231201に含まれるものについては、target3を1にする
test.loc[test["original_id"].isin(closed_in_3month_from20231201),"target3"]=1


test[save_columns].to_pickle(data_folda+'skeleton_test.pkl')

* Train/Test 期間のeval_periodの数を確認

In [124]:
test["eval_period"].value_counts()

True    844322
Name: eval_period, dtype: int64

In [125]:
for d in pseudo_time_series["base_date"].unique():
    print(d,pseudo_time_series[(pseudo_time_series["base_date"]==d)
                    &(pseudo_time_series["eval_period"]==True)]["eval_period"].value_counts())

2018-12-31T00:00:00.000000000 True    605825
Name: eval_period, dtype: int64
2019-01-31T00:00:00.000000000 True    608934
Name: eval_period, dtype: int64
2019-02-28T00:00:00.000000000 True    612213
Name: eval_period, dtype: int64
2019-03-31T00:00:00.000000000 True    616191
Name: eval_period, dtype: int64
2019-04-30T00:00:00.000000000 True    620309
Name: eval_period, dtype: int64
2019-05-31T00:00:00.000000000 True    624541
Name: eval_period, dtype: int64
2019-06-30T00:00:00.000000000 True    628699
Name: eval_period, dtype: int64
2019-07-31T00:00:00.000000000 True    632987
Name: eval_period, dtype: int64
2019-08-31T00:00:00.000000000 True    637478
Name: eval_period, dtype: int64
2019-09-30T00:00:00.000000000 True    641904
Name: eval_period, dtype: int64
2019-10-31T00:00:00.000000000 True    646336
Name: eval_period, dtype: int64
2019-11-30T00:00:00.000000000 True    651001
Name: eval_period, dtype: int64
2019-12-31T00:00:00.000000000 True    655051
Name: eval_period, dtype: int64

In [126]:
pseudo_time_series.columns

Index(['restaurant_id', 'original_id', 'available', 'closed_date',
       'business_status', 'prefecture', 'id', 'created_at', 'base_date',
       'closed_flg', 'target3', 'target6', 'eval_period'],
      dtype='object')

* Train/Test 期間のtarget3を確認

In [127]:
test["target3"].value_counts()

0    840306
1      4016
Name: target3, dtype: int64

In [128]:
for d in pseudo_time_series["base_date"].unique():
    print(d,pseudo_time_series[(pseudo_time_series["base_date"]==d)
                    &(pseudo_time_series["eval_period"]==True)]["target3"].value_counts())

2018-12-31T00:00:00.000000000 0    605825
Name: target3, dtype: int64
2019-01-31T00:00:00.000000000 0    608934
Name: target3, dtype: int64
2019-02-28T00:00:00.000000000 0    612213
Name: target3, dtype: int64
2019-03-31T00:00:00.000000000 0    616191
Name: target3, dtype: int64
2019-04-30T00:00:00.000000000 0    620309
Name: target3, dtype: int64
2019-05-31T00:00:00.000000000 0    624541
Name: target3, dtype: int64
2019-06-30T00:00:00.000000000 0    628699
Name: target3, dtype: int64
2019-07-31T00:00:00.000000000 0    632987
Name: target3, dtype: int64
2019-08-31T00:00:00.000000000 0    637478
Name: target3, dtype: int64
2019-09-30T00:00:00.000000000 0    641904
Name: target3, dtype: int64
2019-10-31T00:00:00.000000000 0    646336
Name: target3, dtype: int64
2019-11-30T00:00:00.000000000 0    651001
Name: target3, dtype: int64
2019-12-31T00:00:00.000000000 0    655051
Name: target3, dtype: int64
2020-01-31T00:00:00.000000000 0    659166
Name: target3, dtype: int64
2020-02-29T00:00:00.